In [4]:
# Import the libraries 
import nltk 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
import numpy as np 
import gensim
from gensim.models import Word2Vec
import pandas as pd 

In [5]:
df_data = pd.read_csv("C:/Users/HP/Downloads/PM_Modi_Speech_Text_english.csv") 
#Loading the dataset        

In [6]:
df_data

,Unnamed: 0,Date,Speech,Title
0,2,09 JAN 2017,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...
1,3,09 JAN 2017,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...
2,5,03 JAN 2017,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...
3,6,26 FEB 2017,"My dear countrymen, Namaskar. Winter is on its...",English Translation of the text of PM’s ‘Mann ...
4,8,21 FEB 2017,Namaskaram. Greetings to everyone. Swami Nirvi...,Text of PM’s inaugural address (via video conf...
...,...,...,...,...
1177,1263,08 APR 2023,Bharat Mata Ki Jai!\n\nBharat Mata Ki Jai!\n\n...,English rendering of PM’s address at launch of...
1178,1264,04 APR 2023,ExcellenciesHonourable Minister Mr. Harbers;Sp...,"Text of Address by Dr. P.K. Mishra, Principal ..."
1179,1265,04 APR 2023,"Namaskar!Excellencies, heads of state, Academi...",Text of PM’s remarks at International Conferen...
1180,1266,03 APR 2023,My colleague in the Union Cabinet Dr. Jitendra...,English rendering of PM’S address at the diamo...


In [7]:
df_data = df_data.drop("Unnamed: 0", axis = 1)

In [8]:
from datetime import datetime
month_dict = {
    "JAN": "01",
    "FEB": "02",
    "MAR": "03",
    "APR": "04",
    "MAY": "05",
    "JUN": "06",
    "JUL": "07",
    "AUG": "08",
    "SEP": "09",
    "OCT": "10",
    "NOV": "11",
    "DEC": "12"
}

def dates(x):
    dt = datetime.strptime(x, "%d %b %Y")
    return dt

df_data['Date'] = df_data['Date'].apply(dates)
df_data = df_data.sort_values('Date')


In [9]:
df_data

,Date,Speech,Title
2,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...
0,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...
1,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...
6,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...
5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...
...,...,...,...
1175,2023-04-09,My colleagues in the Union Cabinet Shri Bhupen...,English rendering of PM’s address at the inaug...
1173,2023-04-12,Namaskar! \n\nThe campaign to provide governme...,English rendering of PM’s address at training ...
1174,2023-04-12,"Namaskar, Governor of Rajasthan Shri Kalraj Mi...",English rendering of PM’s address during flagg...
1172,2023-04-14,"Governor of Assam Shri Gulab Chand Kataria ji,...",English rendering of PM’s address at inaugurat...


### Indexing

In [10]:
#changing the index
new_index = ['NMS' + str(i) for i in range(1, len(df_data)+1)]
#renaming to reflect on df
df_data = df_data.rename(index=dict(zip(df_data.index, new_index)))
df_data

,Date,Speech,Title
NMS1,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...
NMS2,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...
NMS3,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...
NMS4,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...
NMS5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...
...,...,...,...
NMS1178,2023-04-09,My colleagues in the Union Cabinet Shri Bhupen...,English rendering of PM’s address at the inaug...
NMS1179,2023-04-12,Namaskar! \n\nThe campaign to provide governme...,English rendering of PM’s address at training ...
NMS1180,2023-04-12,"Namaskar, Governor of Rajasthan Shri Kalraj Mi...",English rendering of PM’s address during flagg...
NMS1181,2023-04-14,"Governor of Assam Shri Gulab Chand Kataria ji,...",English rendering of PM’s address at inaugurat...


In [19]:
# Converts the Speech column to a string data type.
df_data['Speech'] = df_data['Speech'].astype(str)

In [20]:
# Initializing the model 
lemmatizer = WordNetLemmatizer()

In [22]:
def cleaned(speech):
    # Sentence Tokenization
    sentence = nltk.sent_tokenize(speech)

    # Creating an empty list to store the cleaned dataset 
    corpus = []

    # Loop through each sentence in the speech
    for i in range(len(sentence)):
        # Remove any non-alphabetic characters in the sentence and convert to lowercase
        review = re.sub("[^a-zA-Z0-9]", " ", sentence[i])
        review = review.lower()
        
        # Tokenize the sentence into a list of words
        review = review.split()
        
        # Lemmatize each word in the sentence and remove stop words
        lemmatizer = WordNetLemmatizer()
        review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
        
        # Convert the list of words back into a sentence
        review = ' '.join(review)
        
        # Add the cleaned sentence to the corpus
        corpus.append(review)

    return corpus[:-1] # Removing the last element of the list 


# To convert cleaned speeches into string for performing NER
def string(speech):
    string_speech = " ".join(speech)
    return string_speech

# Defining function to convert the corpus into list format 
# def cleaned_list(x):
#     y = x.replace('[','').replace(']','').replace("'","").split(', ')
#     return y 


In [23]:
# Clean the text data in the 'Speech' column of the dataframe using the 'cleaned' function
# and create a new column 'Cleaned_Speech' to store the cleaned text data
df_data['Cleaned_Speech'] = df_data['Speech'].apply(cleaned)

# Convert the cleaned text data in the 'Cleaned_Speech' column to a string 
# using the 'string' function and create a new column 'Cleaned_Speech_string'
# to store the string data
df_data['Cleaned_Speech_string'] = df_data['Cleaned_Speech'].apply(string)

# Convert the corpus into list 
#df_data['Cleaned_Speech'] = df_data['Cleaned_Speech'].apply(cleaned_list)

AttributeError: 'list' object has no attribute 'replace'

In [38]:
df_data = df_data.reset_index()

In [39]:
df_data

,index,Date,Speech,Title,Cleaned_Speech,Cleaned_Speech_string
0,NMS1,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...,[governor andhra pradesh shri e l narasimhan c...,governor andhra pradesh shri e l narasimhan ch...
1,NMS2,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...,[chief minister gujarat shri vijay rupani ji c...,chief minister gujarat shri vijay rupani ji co...
2,NMS3,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...,[delighted gift city inaugurate india first in...,delighted gift city inaugurate india first int...
3,NMS4,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...,[congratulate finance minister arun jaitley je...,congratulate finance minister arun jaitley jee...
4,NMS5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...,[sri pejavar math respected sri vishwesh tirth...,sri pejavar math respected sri vishwesh tirth ...
...,...,...,...,...,...,...
1177,NMS1178,2023-04-09,My colleagues in the Union Cabinet Shri Bhupen...,English rendering of PM’s address at the inaug...,[colleague union cabinet shri bhupender yadav ...,colleague union cabinet shri bhupender yadav j...
1178,NMS1179,2023-04-12,Namaskar! \n\nThe campaign to provide governme...,English rendering of PM’s address at training ...,"[namaskar, campaign provide government job you...",namaskar campaign provide government job youth...
1179,NMS1180,2023-04-12,"Namaskar, Governor of Rajasthan Shri Kalraj Mi...",English rendering of PM’s address during flagg...,[namaskar governor rajasthan shri kalraj mishr...,namaskar governor rajasthan shri kalraj mishra...
1180,NMS1181,2023-04-14,"Governor of Assam Shri Gulab Chand Kataria ji,...",English rendering of PM’s address at inaugurat...,[governor assam shri gulab chand kataria ji ch...,governor assam shri gulab chand kataria ji chi...


In [40]:
# Saving the cleaned data to the csv file 
df_data.to_csv("C:/Users/HP/Downloads/PM_Modi_Speech_Cleaned_string.csv", index = False)

In [ ]:
# Reloading the cleaned data for vectorization 
df_data = pd.read_csv("E:\Tarang\Ashoka\Python\PYTHON PROJECT\PM_Modi_Speech_Cleaned_string.csv")

In [7]:
# Final code 
def vectorize(corpus):

    corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in corpus]
    
    model = Word2Vec(sentences=corpus_tokenized, vector_size=300, window=5, min_count=5, workers=4)

    vectors = []

    for sentence in corpus_tokenized:
        sent_vec = []
        for word in sentence:
            if word in model.wv.key_to_index:
                # If the word is in the Word2Vec model's vocabulary, extract its vector
                word_vector = model.wv.get_vector(word)
                sent_vec.append(word_vector)
        vectors.append(sent_vec)

    return vectors 


In [ ]:
# To convert cleaned speeches into string for performing NER
def string(speech):
    string_speech = " ".join(speech)
    return string_speech

def cleaned_str(x):
    y = x.replace('[','').replace(']','').replace("'","").split(', ')
    return y 


In [ ]:
# Vectorize the cleaned text data in the 'Cleaned_Speech' column
# using the 'vectorize' function and create a new column 'Vectorized_Speech'
# to store the vectorized data
df_data['vectorize'] = df_data['Cleaned_Speech'].apply(vectorize)

In [ ]:
# Creating the DataFrame of each sentence 
for k in range(len(df_data)):
    
    dfs_sen = []

    speech_list = df_data['vector'][k]

    for i in range(len(speech_list)):
        numpy_array = np.array(speech_list[i])
        dfs_sen.append(pd.DataFrame(numpy_array))


    for i, df in enumerate(dfs_sen):
        new_name = f'NMSP{k+1}S{i+1}'  # create new name for dataframe
        globals()[new_name] = df  # assign renamed dataframe to new name
    
    for i, df in enumerate(dfs_sen):
        # Create a new list of index names based on the index of the current DataFrame
        new_index = [f"NMSP{k+1}S{i+1}W{j+1}" for j in range(len(df))]
        # Set the new index for the current DataFrame
        df.index = new_index
    

In [ ]:
# Saving the vectorized speech to the csv file 
df_data.to_csv("E:\Tarang\Ashoka\Python\PYTHON PROJECT\PM_Modi_Speech_Vectorized.csv", index = False)

In [126]:
NMSP3S3

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
NMSP3S3W1,-0.000634,-1.630372e-03,0.003173,-0.003050,0.001496,0.001522,-0.000373,0.003344,0.003316,-0.001891,...,0.002340,0.000841,0.001914,0.002023,0.000796,-0.001800,0.000033,0.002654,-0.001954,0.003055
NMSP3S3W2,0.003287,-3.087554e-03,-0.002170,0.000997,0.002168,-0.002144,0.000974,0.003452,-0.002310,-0.002125,...,-0.002188,0.001337,0.000474,0.002683,0.002473,-0.000632,0.000874,-0.000726,-0.003061,-0.000968
NMSP3S3W3,0.001845,3.561138e-03,-0.002377,-0.000073,-0.001659,0.001765,0.003302,0.000429,0.001779,0.002086,...,-0.001402,-0.002401,-0.002473,0.001082,-0.002927,0.002342,-0.000970,-0.003262,-0.001558,0.002449
NMSP3S3W4,0.003075,-2.549309e-03,0.001230,0.003204,0.002972,0.000343,-0.000497,-0.001818,0.002281,0.000890,...,-0.000798,0.002663,0.002198,0.000401,-0.003045,-0.003180,-0.000155,0.002587,0.002878,-0.001186
NMSP3S3W5,0.001858,2.673474e-03,-0.001312,-0.000999,0.000312,-0.003143,-0.003218,0.003262,-0.002970,0.001027,...,-0.000285,-0.002849,0.001542,-0.003343,0.000342,0.001993,0.002456,-0.000939,-0.003292,0.000214
NMSP3S3W6,-0.001571,-4.801755e-05,0.001120,-0.002010,-0.003280,-0.003714,0.003140,0.002658,-0.001569,-0.002936,...,0.002795,-0.001519,-0.000201,-0.002913,-0.001817,-0.002184,0.001329,0.000599,0.002090,0.000646
NMSP3S3W7,-0.000841,-1.688540e-03,-0.003075,-0.000747,-0.001434,-0.003467,0.001174,-0.001619,0.003135,0.000115,...,-0.002440,-0.002452,0.000270,-0.001588,0.001327,0.003196,0.000147,-0.001217,-0.000852,0.000085
NMSP3S3W8,-0.002414,-2.015501e-03,0.000435,-0.001364,0.002550,-0.000978,0.002716,-0.002755,-0.001723,-0.002505,...,0.000246,-0.002651,-0.000578,0.002964,0.000693,-0.000980,0.003017,-0.002609,0.002082,0.002930
NMSP3S3W9,0.000875,-2.685801e-04,0.000704,-0.002302,0.001961,0.000096,-0.001741,0.001114,-0.000021,0.001755,...,-0.000766,0.002354,-0.000790,-0.003266,-0.001898,-0.001802,-0.001636,0.001887,-0.002272,-0.002643
NMSP3S3W10,-0.002854,3.247109e-03,-0.003086,0.003077,-0.000374,0.000569,0.000108,0.003350,-0.002172,0.002579,...,-0.000136,-0.001593,-0.002061,-0.002850,-0.003011,-0.000481,0.003043,-0.000173,-0.002507,-0.002086


## STANDARDIZATION USING MIN-MAX SCALAR

In [69]:
# Min Max Scaling
np_m_df_norm = (np_m_df-np_m_df.min())/(np_m_df.max()-np_m_df.min())
np_m_df_norm

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.879742,0.085635,0.280214,0.149869,0.410721,0.964471,0.049500,0.020517,0.227270,0.895903,...,0.498700,0.027320,0.049409,0.581606,0.049421,0.030612,0.551777,0.901156,0.069885,0.832883
1,0.836504,0.106268,0.279186,0.189869,0.477076,0.941234,0.079705,0.055203,0.162551,0.879453,...,0.610892,0.057937,0.075616,0.569215,0.124347,0.073970,0.582357,0.856149,0.065231,0.815615
2,0.832703,0.117942,0.111305,0.171239,0.467508,0.932522,0.054544,0.036838,0.282884,0.820145,...,0.510437,0.089846,0.055937,0.551652,0.075254,0.082660,0.458407,0.885343,0.081385,0.843604
3,0.840986,0.106825,0.240669,0.187034,0.490078,0.964111,0.038195,0.026015,0.281973,0.905181,...,0.519874,0.024470,0.060213,0.613723,0.011912,0.036528,0.454286,0.864814,0.077277,0.910023
4,0.821982,0.080235,0.155331,0.139735,0.451723,0.980944,0.005654,0.004388,0.269059,0.881707,...,0.519372,0.013811,0.027030,0.576257,0.020236,0.013313,0.466610,0.906133,0.022629,0.918730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,0.830493,0.115558,0.259110,0.248762,0.392465,0.924909,0.076717,0.070264,0.217646,0.770051,...,0.580971,0.090674,0.074362,0.571345,0.107182,0.093079,0.540789,0.870597,0.105248,0.802517
1178,0.873879,0.078034,0.205670,0.175532,0.512358,0.990566,0.017361,0.012375,0.147338,0.938298,...,0.560101,0.006802,0.041853,0.535858,0.022217,0.030894,0.537090,0.885977,0.038622,0.887289
1179,0.865288,0.086543,0.197648,0.180101,0.400704,0.972449,0.048958,0.024447,0.168668,0.834417,...,0.576484,0.033808,0.019456,0.569702,0.046682,0.028150,0.504341,0.876764,0.084780,0.831694
1180,0.821766,0.077706,0.203304,0.153766,0.470526,0.961035,0.014116,0.020693,0.270386,0.890261,...,0.580202,0.033275,0.053359,0.595481,0.051605,0.046055,0.455575,0.908770,0.045537,0.887658
